In [ ]:
import pdb
import pickle
import string

import time

import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import scipy
import sklearn
from gensim.models import KeyedVectors
from nltk.corpus import stopwords, twitter_samples
from nltk.tokenize import TweetTokenizer

from utils import (cosine_similarity, get_dict,
                   process_tweet)
from os import getcwd

In [ ]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [ ]:
from gensim.models import KeyedVectors

en_embeddings = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary = True)
fr_embeddings = KeyedVectors.load_word2vec_format('./wiki.multi.fr.vec')


en_fr_train = get_dict('en-fr.train.txt')
print('The length of the english to french training dictionary is', len(en_fr_train))
en_fr_test = get_dict('en-fr.test.txt')
print('The length of the english to french test dictionary is', len(en_fr_train))

english_set = set(en_embeddings.vocab)
french_set = set(fr_embeddings.vocab)
en_embeddings_subset = {}
fr_embeddings_subset = {}
french_words = set(en_fr_train.values())

for en_word in en_fr_train.keys():
    fr_word = en_fr_train[en_word]
    if fr_word in french_set and en_word in english_set:
        en_embeddings_subset[en_word] = en_embeddings[en_word]
        fr_embeddings_subset[fr_word] = fr_embeddings[fr_word]


for en_word in en_fr_test.keys():
    fr_word = en_fr_test[en_word]
    if fr_word in french_set and en_word in english_set:
        en_embeddings_subset[en_word] = en_embeddings[en_word]
        fr_embeddings_subset[fr_word] = fr_embeddings[fr_word]


pickle.dump( en_embeddings_subset, open( "en_embeddings.p", "wb" ) )
pickle.dump( fr_embeddings_subset, open( "fr_embeddings.p", "wb" ) )


In [ ]:
en_embeddings_subset = pickle.load(open("en_embeddings.p", "rb"))
fr_embeddings_subset = pickle.load(open("fr_embeddings.p", "rb"))

In [ ]:
en_fr_train = get_dict('en-fr.train.txt')
print('The length of the English to French training dictionary is', len(en_fr_train))
en_fr_test = get_dict('en-fr.test.txt')
print('The length of the English to French test dictionary is', len(en_fr_train))

In [ ]:
def get_matrices(en_fr, french_vecs, english_vecs):
    X_l = list()
    Y_l = list()

    english_set = english_vecs.keys()

    french_set = french_vecs.keys()

    french_words = set(en_fr.values())

    for en_word, fr_word in en_fr.items():

        if fr_word in french_set and en_word in english_set:

            en_vec = english_vecs[en_word]

            fr_vec = french_vecs[fr_word]

            X_l.append(en_vec)

            Y_l.append(fr_vec)

    X =  np.vstack(X_l)

    Y = np.vstack(Y_l)
    
    return X, Y


In [ ]:
X_train, Y_train = get_matrices(
    en_fr_train, fr_embeddings_subset, en_embeddings_subset)

In [ ]:
def compute_loss(X, Y, R):
    m = X.shape[0]
    
    diff = np.dot(X,R)-Y

    diff_squared = diff**2

    sum_diff_squared = np.sum(diff_squared)

    loss = sum_diff_squared/m
    return loss

In [ ]:
def compute_gradient(X, Y, R):
    m = X.shape[0]

    gradient = np.dot(X.transpose(),np.dot(X,R)-Y)*(2/m)
    return gradient

In [ ]:
def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003):
    np.random.seed(129)
    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if i % 25 == 0:
            print(f"loss at iteration {i} is: {compute_loss(X, Y, R):.4f}")
        gradient = compute_gradient(X,Y,R)

        R -=  learning_rate * gradient
    return R

In [ ]:
np.random.seed(129)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * .1
R = align_embeddings(X, Y)

In [ ]:
R_train = align_embeddings(X_train, Y_train, train_steps=400, learning_rate=0.8)

In [ ]:
def nearest_neighbor(v, candidates, k=1):
    similarity_l = []

    for row in candidates:
        cos_similarity = cosine_similarity(v,row)

        similarity_l.append(cos_similarity)
        
    sorted_ids = np.argsort(similarity_l)

    k_idx = sorted_ids[-k:]
    return k_idx

In [ ]:
v = np.array([1, 0, 1])
candidates = np.array([[1, 0, 5], [-2, 5, 3], [2, 0, 1], [6, -9, 5], [9, 9, 9]])
print(candidates[nearest_neighbor(v, candidates, 3)])

In [ ]:
def test_vocabulary(X, Y, R):
    pred = np.dot(X,R)

    num_correct = 0

    for i in range(len(pred)):
        pred_idx = nearest_neighbor(pred[i],Y)

        if pred_idx == i:
            num_correct += 1

    accuracy = num_correct / len(pred)
    
    return accuracy

In [ ]:
X_val, Y_val = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

In [ ]:
acc = test_vocabulary(X_val, Y_val, R_train)  # this might take a minute or two
print(f"accuracy on test set is {acc:.3f}")